# Whale Pivotality in Aave top Snapshot proposals

In [1]:
# sets up the pynb environment
from datetime import datetime
from zoneinfo import ZoneInfo
import os
import sys

from IPython.display import HTML
import pandas as pd

from apis.dune import get_top_uniswap_proposals

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from stages.dataframe_filters.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)

In [2]:
print(f"Last updated on {datetime.now(ZoneInfo('UTC')).date()}")

Last updated on 2023-05-20


In [3]:
all_uniswap_proposals = pd.read_csv(
    "../plutocracy_data/full_report/Uniswap_report.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)
all_uniswap_proposals_filtered = pd.read_csv(
    "../plutocracy_data/full_report/Uniswap_report_filtered.csv.gzip",
    engine="c",
    low_memory=False,
    compression='gzip',
)


In [4]:
def to_organization_map(flat_organization_dataframe: pd.DataFrame):
    return {
        organization_name: proposal_df
        for organization_name, proposal_df in [
            (str(organization_name), space_proposals)
            for organization_name, space_proposals in flat_organization_dataframe.groupby(
                "proposal_organization_name"
            )
        ]
    }

plutocracy_report_data = to_organization_map(all_uniswap_proposals)
plutocracy_report_data_filtered = to_organization_map(all_uniswap_proposals_filtered)


In [5]:
pd.set_option("display.max_rows", int(1e3))
score_differences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)

In [6]:
score_differences_dfs: dict[str, pd.DataFrame] = dict()

initial_series_data = {
    organization: 0
    for organization in plutocracy_report_data.keys()
}
changed_outcome_proportions = pd.Series(initial_series_data, name="changed outcomes %")

for score_difference in score_differences:
    for organization, data in score_difference.items():
        data: dict[str, list] = data
        items = data.items()
        score_differences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([id for id, _ in items]), name="Proposal"
            ),
            columns=[
                "proposal_id",
                "Title",
                "Date Started",
                "Date Ended",
                "score_differences",
                "whale_vp_proportion",
                "total_vp",
                "outcome_changed",
                "Result",
                "Result w/o Whales"
            ],
        ).astype({"total_vp": "float64"}, copy=False
        ).sort_values(["total_vp"], ascending=False)


        changed_outcome_proportions[organization] = score_differences_dfs[organization]["outcome_changed"].value_counts(normalize=True)[True]

        score_differences_dfs[organization].drop("outcome_changed", axis=1, inplace=True)

        score_differences_dfs[organization]["Date Started"] = pd.to_datetime(score_differences_dfs[organization]["Date Started"], utc=True, unit="s").dt.strftime("%d/%m/%y")
        score_differences_dfs[organization]["Date Ended"] = pd.to_datetime(score_differences_dfs[organization]["Date Ended"], utc=True, unit="s").dt.strftime("%d/%m/%y")

        organization_id = plutocracy_report_data[organization].iloc[0]["proposal_organization_id"]

        score_differences_dfs[organization]["total_vp"] = score_differences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)

        score_differences_dfs[organization]["Title"] = score_differences_dfs[organization].apply(
            lambda row: f'<a href=http://snapshot.org/#/{organization_id}/proposal/{row.name} rel="noopener noreferrer" target="_blank">{row["Title"]}</a>',
            axis=1
        )
        score_differences_dfs[organization].style.format({"whale_vp_proportion": "{:.2%}".format})
changed_outcome_proportions = changed_outcome_proportions.apply(
    lambda proportion: "{:.2%}".format(proportion)
)

In [7]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


## Synthesis

Show the percentage of proposals whose outcome changes if whales did not vote (i.e. *whale pivotality*).

In [8]:
dao_overview = pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
    ],
)
dao_overview.set_index(
    pd.Index([list(result.items())[0][0] for result in voting_ratios], name="DAO"),
    inplace=True
)

dao_overview.insert(2, changed_outcome_proportions.name, changed_outcome_proportions)
dao_overview

,# of whales,all voters,changed outcomes %
DAO,,,
Uniswap,783,24558,16.47%


In [9]:
top_10_proposals = get_top_uniswap_proposals()
top_10_proposals_df = pd.DataFrame(top_10_proposals)

## Comparison of the Top 10 Proposals

In [10]:
top_10_proposal_ids = top_10_proposals_df["proposal_id"].values

mask = score_differences_dfs["Uniswap"].index.to_series().apply(
    lambda row: row in top_10_proposal_ids
)

score_differences_dfs["Uniswap"] = score_differences_dfs["Uniswap"].loc[mask]
proposal_ids = score_differences_dfs["Uniswap"]["proposal_id"]
proposal_ids = pd.Categorical(proposal_ids, top_10_proposal_ids)

score_differences_dfs["Uniswap"] = score_differences_dfs["Uniswap"].loc[proposal_ids]
score_differences_dfs["Uniswap"]["proposal_id"] = proposal_ids
score_differences_dfs["Uniswap"] = score_differences_dfs["Uniswap"].loc[proposal_ids].sort_values("proposal_id")

HTML(
    score_differences_dfs["Uniswap"]
    .rename(lambda index: score_differences_dfs["Uniswap"].loc[index]["Title"])
    .drop(["score_differences", "total_vp", "whale_vp_proportion", "Title", "proposal_id"], axis=1)
    .to_html(render_links=True, escape=False)
)

,Date Started,Date Ended,Result,Result w/o Whales
Proposal,,,,
Community Governance Process Changes,14/12/22,21/12/22,Yes - Enact changes,Yes - Enact changes
[Temperature Check] Which bridge should Uniswap v3 use for cross-chain governance messaging between Ethereum and BNB Chain?,27/01/23,31/01/23,Wormhole,Wormhole
[Temperature Check] Enable 1bp Fee Tier for UniswapV3 on Arbitrum,02/02/23,07/02/23,For,For
[Temperature Check] Post-BSL cross-chain deployment process and creation of new uniswap.eth subdomain,06/04/23,13/04/23,Yes - create new subdomain,Yes - create new subdomain
Temperature Check: Create Accoutability Committee,24/02/23,03/03/23,Doo Wan Nam (StableLab),Cam O'Donnell (Consensys)
Uniswap V3 Launch on zkEVM,29/03/23,03/04/23,Yes,Yes
[Temperature Check] Deploy Uniswap v3 on Avalanche,04/03/23,09/03/23,Yes,Yes
[Temperature Check] Should Uniswap v3 be deployed to BNB Chain? 🦄,17/01/23,22/01/23,Yes,Yes
[Temperature Check] Deploy Uniswap v3 on Moonbeam (2023),25/04/23,30/04/23,Yes,Yes


<sup>Source for proposal list: https://dune.com/queries/2488751</sup>

### Proportion of Outcomes Changed

In [11]:
print(f"{changed_outcome_proportions['Uniswap']} of Uniswap's proposal outcomes (of the 150 sampled) change after filtering out whale voting power.")

16.47% of Uniswap's proposal outcomes (of the 150 sampled) change after filtering out whale voting power.


---